### Explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def sizeOfData():
    
    
    return spam_data['target'].mean()*100

In [4]:
sizeOfData()

13.406317300789663

In [5]:
# longest token in the vocabulary

from sklearn.feature_extraction.text import CountVectorizer

def longestToken():
    vect = CountVectorizer().fit(X_train)
    vocab = vect.get_feature_names()
    length = [len(w) for w in vocab]
    
    return vocab[length.index(max(length))]
    

In [6]:
longestToken()

'com1win150ppmx3age16subscription'

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def NBmodel():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)


    
    
    return roc_auc_score(y_test, predictions)

In [8]:
NBModel()

0.97208121827411165

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TFIDF():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    feature_names = np.array(vect.get_feature_names())
    #idfs = vect.idf_     
    #sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
    values =  X_train_vectorized.max(0).toarray()[0]
    index = vect.get_feature_names()
    
    features_series = pd.Series(values,index = index)
    
    
    #largest= sorted(names_idfs, key=operator.itemgetter(1),reverse=True)[:20]

#     print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
#     print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))
    
    return  features_series.nsmallest(20),features_series.nlargest(20)

In [15]:
TFIDF()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

In [10]:
def TFIDFNB():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)

    
    
    return roc_auc_score(y_test, predictions)

In [11]:
TFIDFNB()

0.94162436548223349

### Does the length play a role in spam vs non-spam?

What is the average length of documents (number of characters) for not spam and spam documents?


In [12]:
def charLength():
    spamNumber = spam_data['target'].sum()
    total = len(spam_data)
    
    
    spam_data['length']=spam_data['text'].apply(lambda x:len(x))
    noSpamChar = sum(spam_data['length'][spam_data['target'] == 0])
    spamChar   = sum(spam_data['length'][spam_data['target'] == 1])
    
    
    return (noSpamChar/(total-spamNumber), spamChar/spamNumber ) 

In [13]:
charLength()

(71.023626943005183, 138.8661311914324)

In [14]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### AUC with SVM 


In [16]:
from sklearn.svm import SVC

def SVMModel():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    X_train_wLength = add_feature(X_train_vectorized, X_train.str.len())
    
    X_test_vectorized = vect.transform(X_test)
    X_test_wLength  = add_feature(X_test_vectorized, X_test.str.len())

    
    clfrSVM = SVC(C=10000)
    clfrSVM.fit(X_train_wLength,y_train)
    predictions= clfrSVM.predict(X_test_wLength)
    
    
    
    
    return roc_auc_score(y_test, predictions)

In [17]:
SVMModel()

0.95813668234215565

### number of digits per document


In [18]:
def nDigits():
    spamNumber = spam_data['target'].sum()
    total = len(spam_data)
    
    
    spam_data['ndig']=spam_data['text'].apply(lambda x: sum(c.isdigit() for c in x))
    noSpamChar = sum(spam_data['ndig'][spam_data['target'] == 0])
    spamChar   = sum(spam_data['ndig'][spam_data['target'] == 1])
    
    
    return (noSpamChar/(total-spamNumber), spamChar/spamNumber ) 
    

In [19]:
nDigits()

(0.29927461139896372, 15.759036144578314)

### N-gram?

In [20]:
from sklearn.linear_model import LogisticRegression

def spNgram():
    

    vectorizer = TfidfVectorizer(min_df=5, ngram_range=[1,3])

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_with_length = add_feature(X_train_transformed, [X_train.str.len(),
                                                                        X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_with_length = add_feature(X_test_transformed, [X_test.str.len(),
                                                                      X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])


    
    model = LogisticRegression(C=100)
    model.fit(X_train_transformed_with_length,y_train)
    predictions= model.predict(X_test_transformed_with_length)
    
    
    
    
    return roc_auc_score(y_test, predictions)

In [21]:
spNgram()

0.96787090640544626

### non-word characters?


In [22]:
def nonWord():
    spamNumber = spam_data['target'].sum()
    total = len(spam_data)
    
    
    spam_data['nonWord']=spam_data['text'].str.count(r'\W')
    noSpamChar = sum(spam_data['nonWord'][spam_data['target'] == 0])
    spamChar   = sum(spam_data['nonWord'][spam_data['target'] == 1])
    
    
    return (noSpamChar/(total-spamNumber), spamChar/spamNumber ) 

In [23]:
nonWord()

(17.291813471502589, 29.041499330655956)

### Logit

Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

In [24]:
def SpamLogit():
    #vectorizer = TfidfVectorizer(analyzer='char_wb', min_df=5, ngram_range=(2,5))
    vectorizer =CountVectorizer(analyzer='char_wb', min_df=5, ngram_range=(2,5))

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_with_length = add_feature(X_train_transformed, [X_train.str.len(),
                                                                        X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                                       X_train.str.findall(r'(\W)').str.len()])

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_with_length = add_feature(X_test_transformed, [X_test.str.len(),
                                                                      X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                                     X_test.str.findall(r'(\W)').str.len()])


    
    model = LogisticRegression(C=100)
    model.fit(X_train_transformed_with_length,y_train)
    predictions= model.predict(X_test_transformed_with_length)
    
    
    feature_names = np.array(vectorizer.get_feature_names()+ ['length', 'digit', 'non_word'])
    
    sorted_coef_index = model.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]].tolist()
    largest = feature_names[sorted_coef_index[:-11:-1]].tolist()
    
    
    return roc_auc_score(y_test, predictions),smallest,largest


In [25]:
spamLogit()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])